In [1]:
# Import necessary modules
import sys
sys.path.append("..")
from src.confpickem.yahoo_pickem_scraper import YahooPickEm, calculate_player_stats
from src.confpickem.confidence_pickem_sim import ConfidencePickEmSimulator, Player
from src.confpickem.yahoo_pickem_integration import convert_yahoo_to_simulator_format, convert_yahoo_picks_to_dataframe
import pandas as pd

In [2]:
# Pull data from Yahoo
week = 18
league_id = 6207
num_sims = 1000
# num_entries = 50

yahoo_data = YahooPickEm(week, league_id, cookies_file='../src/confpickem/cookies.txt')
# yahoo_data.games
# yahoo_data.players
# yahoo_data.results

In [3]:
# Initialize simulator instance and feed in NFL game details
simulator = ConfidencePickEmSimulator(num_sims=num_sims)
games_df = convert_yahoo_to_simulator_format(yahoo_data, ignore_results=False)#True)
simulator.add_games_from_dataframe(games_df)

# games_df
# simulator.games

In [4]:
# Create average players and feed into simulator
# simulator.players = [
#         Player(
#             name=f"Player {ind + 1}",
#             skill_level=0.749,
#             crowd_following=0.737,
#             confidence_following=0.826
#         ) for ind in range(num_entries)
#     ]

# Calculate player statistics from previous results
stats = calculate_player_stats(
        league_id=6207,
        weeks=range(1, week),
        cookies_file='../src/confpickem/cookies.txt'
    )
stats['me'] = stats.player == "Firman's Educated Guesses"
stats = stats.sort_values(by="me",ascending=False,ignore_index=True)

player_stats = [
        Player(
            name=stats.iloc[ind].player,
            skill_level=stats.iloc[ind].skill_level,
            crowd_following=stats.iloc[ind].crowd_following,
            confidence_following=stats.iloc[ind].confidence_following
        ) for ind in range(stats.shape[0])
    ]
simulator.players = player_stats

# simulator.players

In [5]:
# Run simulation with random picks and outcomes
picks_df = simulator.simulate_picks()
outcomes = simulator.simulate_outcomes()
# picks_df.groupby(["player","game","picked_home"]).size().sort_values(ascending=False)
# outcomes.sum(axis=0)

# Analyze results
stats_orig = simulator.analyze_results(picks_df, outcomes)
for stat in stats_orig:
    stats_orig[stat] = stats_orig[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})

summary_orig = pd.merge(left=stats_orig["win_pct"],right=stats_orig["expected_points"],how="inner",on="player")
summary_orig = pd.merge(left=summary_orig,right=stats_orig["point_std"],how="inner",on="player")
summary_orig = pd.merge(left=summary_orig,right=stats_orig["value_at_risk"],how="inner",on="player")
summary_orig.sort_values(by="win_pct",ascending=False)

,player,win_pct,expected_points,point_std,value_at_risk
43,RICHDOG !!!!!!,0.031,76.8165,19.565809,46.975
36,NoWayIGotMorePointsThenYou?!?!,0.029,73.1335,19.806267,41.500
39,OneNDone,0.029,76.7050,19.048906,45.000
4,Ballbusters,0.028,73.2175,18.904916,43.475
3,BFense,0.027,77.1940,18.850111,47.000
23,Hennything Possible,0.026,77.6530,19.319157,46.000
33,Mickey's Team,0.024,74.9930,19.261931,43.000
53,thol1572,0.023,76.0095,18.544839,46.000
8,Boats n Hoes,0.023,75.4805,18.524620,45.975
21,GOOD,0.023,72.9280,19.627737,40.975


In [6]:
# Identify optimal picks for the week
optimal_picks = simulator.optimize_picks("Firman's Educated Guesses",fixed_picks={"Firman's Educated Guesses":{"Bal":16,"Cin":3,"TB":15}})

fixed_picks = {"Firman's Educated Guesses": optimal_picks}

Game(home_team='KC', away_team='Den', vegas_win_prob=0.8255, crowd_home_pick_pct=0.25, crowd_home_confidence=5.4, crowd_away_confidence=7.1, week=18, kickoff_time=Timestamp('2024-01-05 16:25:00-0518', tz='UTC-05:18'), actual_outcome=None, picks_locked=False)


KeyboardInterrupt: 

In [ ]:
# # Setting my picks to the optimized values
# mask = (picks_df.player == simulator.players[0].name)
# for team, pts in optimal_picks.items():
#     game_mask = picks_df[mask].game.str.contains(team)
#     is_home = (picks_df.loc[mask & game_mask, 'game'].str.split('@').str[1] == team).astype(float)
#     picks_df.loc[mask & game_mask, 'picked_home'] = is_home
#     picks_df.loc[mask & game_mask, 'confidence'] = float(pts)

# Run simulation with optimized picks for my team
picks_df = simulator.simulate_picks(fixed_picks)
outcomes = simulator.simulate_outcomes()

# Analyze results
stats_new = simulator.analyze_results(picks_df, outcomes)
for stat in stats_new:
    stats_new[stat] = stats_new[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
summary_new = pd.merge(left=stats_new["win_pct"],right=stats_new["expected_points"],how="inner",on="player")
summary_new = pd.merge(left=summary_new,right=stats_new["point_std"],how="inner",on="player")
summary_new = pd.merge(left=summary_new,right=stats_new["value_at_risk"],how="inner",on="player")
summary_new.sort_values(by="win_pct",ascending=False)


,player,win_pct,expected_points,point_std,value_at_risk
19,Firman's Educated Guesses,0.081,89.5900,14.755189,65.000
7,BigShroomSecurity,0.036,72.9775,18.629084,42.000
12,Caking,0.030,73.6245,19.413736,43.975
25,John K,0.028,73.9140,19.063907,42.000
30,MDTwinTeam,0.027,72.6325,18.526871,42.000
0,#FlytheW,0.025,71.4870,19.019663,40.000
50,Victorious Bastard,0.025,69.8175,19.841538,37.000
6,Big Daddy125,0.025,71.6400,19.224437,37.475
29,Lenzen,0.025,71.5865,18.930614,40.000
2,Arcedinero,0.024,70.8980,19.272442,39.950


In [7]:
# Pulling actual picks from Yahoo
# picks_df_actual = convert_yahoo_picks_to_dataframe(yahoo_data, num_sims)#, optimal_picks)
picks_df_actual = convert_yahoo_picks_to_dataframe(yahoo_data, num_sims)#, fixed_picks)
print(picks_df_actual.loc[picks_df_actual.player == "Firman's Educated Guesses",["game","picked_home"]].drop_duplicates())
# picks_df.picked_home.unique()

# Simulating win probabilities given actual picks
outcomes = simulator.simulate_outcomes()
stats_actual = simulator.analyze_results(picks_df_actual, outcomes)
for stat in stats_actual:
    stats_actual[stat] = stats_actual[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
summary_actual = pd.merge(left=stats_actual["win_pct"],right=stats_actual["expected_points"],how="inner",on="player")
summary_actual = pd.merge(left=summary_actual,right=stats_actual["point_std"],how="inner",on="player")
summary_actual = pd.merge(left=summary_actual,right=stats_actual["value_at_risk"],how="inner",on="player")
summary_actual.sort_values(by="win_pct",ascending=False)

           game  picked_home
192000  Bal@Cle        False
193000  Pit@Cin         True
194000  Atl@Car         True
195000  Dal@Was         True
196000   GB@Chi         True
197000  Ten@Hou        False
198000  Ind@Jax         True
199000   NE@Buf        False
200000  Phi@NYG         True
201000    TB@NO        False
202000   Den@KC         True
203000   LV@LAC         True
204000  LAR@Sea         True
205000  NYJ@Mia        False
206000   Ari@SF         True
207000  Det@Min        False


,player,win_pct,expected_points,point_std,value_at_risk
18,Firman's Educated Guesses,0.492,102.909,10.586538,83.00
26,KKRAHN21,0.126,93.761,10.159779,78.00
25,Johnsonmaia06,0.072,93.791,7.516103,80.00
31,Matt,0.060,86.365,11.416747,66.00
15,Diddypicks,0.053,94.487,7.102131,82.00
12,California Dreaming,0.053,87.178,10.364639,72.00
49,Weird,0.030,88.821,8.517130,76.00
2,Arcedinero,0.023,87.806,9.795014,74.00
9,Boldz36,0.021,84.671,11.284862,64.00
28,Lenzen,0.018,76.908,13.366832,55.00


In [10]:
# Identifying relative importance of each game
importance = simulator.assess_game_importance("Firman's Educated Guesses", picks_df_actual)
print("\nGame Importance Analysis:")
print(importance[["game","points_bid","win_probability","loss_probability","total_impact"]].sort_values('total_impact', ascending=False).to_string(index=False))


Game Importance Analysis:
   game  points_bid  win_probability  loss_probability  total_impact
 Den@KC        14.0            0.625             0.041         0.584
Atl@Car        12.0            0.526             0.000         0.526
 GB@Chi        11.0            0.526             0.000         0.526
LAR@Sea        10.0            0.665             0.223         0.442
 NE@Buf         4.0            0.526             0.119         0.407
 Ari@SF         8.0            0.660             0.274         0.386
Det@Min         5.0            0.729             0.355         0.374
Ind@Jax         9.0            0.835             0.526         0.309
Phi@NYG         6.0            0.740             0.526         0.214
Ten@Hou         1.0            0.625             0.526         0.099
  TB@NO        15.0            0.526             0.433         0.093
 LV@LAC        13.0            0.547             0.478         0.069
NYJ@Mia         2.0            0.565             0.496         0.069
Dal@Was

In [10]:
# # Checking who actually won
# simulator = ConfidencePickEmSimulator(num_sims=num_sims)
# games_df = convert_yahoo_to_simulator_format(yahoo_data, ignore_results=False)
# simulator.add_games_from_dataframe(games_df)
# outcomes = simulator.simulate_outcomes()

# stats_actual = simulator.analyze_results(picks_df_actual, outcomes)
# for stat in stats_actual:
#     stats_actual[stat] = stats_actual[stat].to_frame(stat).reset_index().rename(columns={"index":"player"})
# summary_actual = pd.merge(left=stats_actual["win_pct"],right=stats_actual["expected_points"],how="inner",on="player")
# summary_actual = pd.merge(left=summary_actual,right=stats_actual["point_std"],how="inner",on="player")
# summary_actual = pd.merge(left=summary_actual,right=stats_actual["value_at_risk"],how="inner",on="player")
# summary_actual.sort_values(by=["win_pct","expected_points"],ascending=False)

In [9]:
outcomes.sum(axis=0)

array([   0., 1000., 1000., 1000., 1000., 1000.,    0.,    0.,    0.,
          0.,  806.,  692.,  672.,  537.,  621.,  597.])